<a href="https://colab.research.google.com/github/saif90834/TASK-SOLUTIONS/blob/main/task_27_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from io import StringIO

### Problem-1:

You are given a SQL file link: https://drive.google.com/file/d/1WFt7B84LTHhMueoKmz8W-PRo7xXqmZf3/view?usp=share_link. Read the data by using the file and store it in a excel file. In this data, there are 3 tables named "invoices", "order_leads" and "sales_sql". So create 3 sheets to your excel file.

In [ ]:
# code here
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(host="localhost",user="root",password="",database="temp_db")

invoices = pd.read_sql_query('SELECT * FROM invoices',conn)
order_lead = pd.read_sql_query('SELECT * FROM order_leads',conn)
sales_sql = pd.read_sql_query('SELECT * FROM sales_sql',conn)


with pd.ExcelWriter('supermarket.xlsx') as writer:
    invoices.to_excel(writer,sheet_name="invoices")
    order_lead.to_excel(writer,sheet_name="order_lead")
    sales_sql.to_excel(writer,sheet_name="sales_sql")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 79.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp38-cp38-linux_x86_64.whl size=247965 sha256=897a6a58f1ce532548feb2e5ac2d011c6f4249e74843229071ec847ac4cecb0d
  Stored in directory: /root/.cache/pip/wheels/57/e4/98/5feafb5c393dd2540e44b064a6f95832990d543e5b4f53ea8f
Successfully built mysql.connector


### Problem-2

Go to the site: https://rapidapi.com/wirefreethought/api/geodb-cities. From here, you have to grab the API and have to choose proper routes to get the cities of different countries. After getting the right API, hit that API and create a dataframe of all the cities that you can get by using the API. Then store the dataframe to a SQL. If you need to create an account or have to subscribe, then do that (it has free subscription but has some limitations. Use that free subscription and modify your accordingly to get all the data).  

In [ ]:
### code here
### Rapid API
import requests

url = "https://wft-geo-db.p.rapidapi.com/v1/geo/adminDivisions"

headers = {
    "X-RapidAPI-Key": "b0f75c828fmsh2b0480a62ab8d23p18178ajsn3267e033ae6a",
    "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

data = response.json()['data']
links = response.json()['links']
metadata = response.json()['metadata']

In [ ]:
next_url = links[1]['href']

In [ ]:
count = metadata['totalCount']//5

In [ ]:
import time


for i in range(1,count):

    num_iters += 1

    url = "https://wft-geo-db.p.rapidapi.com"
    page = url + next_url

    response = requests.request("GET", page, headers=headers)

    print(response.json())

    links = response.json()['links']

    for item in links:
        if item['rel'] == 'next':
            next_url = item['href']

    data.extend(response.json()['data'])

    time.sleep(2)


In [ ]:
cities = pd.DataFrame(data)

In [ ]:
import pymysql

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/temp_db")

cities.to_sql("cities",con=engine,if_exists='append')

### Problem 3:

Go to this url: https://www.flipkart.com/search?q=smartphones. This is the url to find phones in flipkart website. You have to extract the below things:
1. image url of the phone
2. name of the image
3. average ratings
4. total ratings
5. total reviews
6. discounted price
7. actual price

Extract all the phones which are available in this website. So you have to use the pagination concept. **Also after requesting every page through the url, please wait for a while (minimum 2-3 seconds), otherwise your IP address can be banned to access the flipkart website later.**

After collecting all the data, save that in a JSON file.

In [ ]:
# code here
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np

In [ ]:
webpage = requests.get('https://www.flipkart.com/search?q=smartphones&page=1').text

In [ ]:
soup = BeautifulSoup(webpage,'lxml')

In [ ]:
all_divs = soup.find_all('div', class_="_2kHMtA")

In [ ]:
phone_image_url = []
phone_name = []
phone_rating = []
phone_num_ratings = []
phone_price = []

for phone in all_divs:
  try:
    phone_image_url.append(phone.find("img",class_="_396cs4")['src'])
  except:
    phone_image_url.append(np.nan)

  try:
    phone_name.append(phone.find("div",class_="_4rR01T").text.strip())
  except:
    phone_name.append(np.nan)

  try:
    phone_rating.append(phone.find("div",class_="_3LWZlK").text.strip())
  except:
    phone_rating.append(np.nan)

  try:
    phone_num_ratings.append(phone.find_all('span',class_='_2_R_DZ')[0].find_all('span')[0].find_all('span')[0].text.strip())
  except:
    phone_num_ratings.append(np.nan)

  try:
    phone_price.append(phone.find("div",class_="_30jeq3 _1_WHN1").text.strip())
  except:
    phone_price.append(np.nan)


df = pd.DataFrame({'phone_name':phone_name,'phone_price':phone_price,'phone_rating':phone_rating,'phone_num_rating':phone_num_ratings,'phone_img_url':phone_image_url})
df

,phone_name,phone_price,phone_rating,phone_num_rating,phone_img_url
0,"Infinix HOT 20 Play (Fantasy Purple, 64 GB)","₹8,199",4.4,"5,321 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
1,"Infinix HOT 20 Play (Racing Black, 64 GB)","₹8,199",4.4,"5,321 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
2,"Infinix HOT 20 Play (Aurora Green, 64 GB)","₹8,199",4.4,"5,321 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
3,"realme C33 (Sandy Gold, 32 GB)","₹8,999",4.4,"17,759 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
4,"Infinix HOT 20 Play (Luna Blue, 64 GB)","₹8,199",4.4,"5,321 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
5,"realme C33 (Night Sea, 32 GB)","₹8,999",4.4,"17,759 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
6,"vivo T1 44W (Midnight Galaxy, 128 GB)","₹14,499",4.5,"82,233 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
7,"vivo T1 44W (Starry Sky, 128 GB)","₹14,499",4.5,"82,233 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
8,"Infinix Smart 6 HD (Aqua Sky, 32 GB)","₹5,599",4.4,"19,552 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
9,"Infinix Smart 6 HD (Origin Blue, 32 GB)","₹5,599",4.4,"19,552 Ratings",https://rukminim1.flixcart.com/image/312/312/x...


In [ ]:
final_df = pd.DataFrame()

for i in range(1,6):

  webpage = requests.get('https://www.flipkart.com/search?q=smartphones&page={}'.format(i)).text
  soup = BeautifulSoup(webpage,'lxml')
  all_divs = soup.find_all('div', class_="_2kHMtA")

  phone_image_url = []
  phone_name = []
  phone_rating = []
  phone_num_ratings = []
  phone_price = []

  for phone in all_divs:
    try:
      phone_image_url.append(phone.find("img",class_="_396cs4")['src'])
    except:
      phone_image_url.append(np.nan)

    try:
      phone_name.append(phone.find("div",class_="_4rR01T").text.strip())
    except:
      phone_name.append(np.nan)

    try:
      phone_rating.append(phone.find("div",class_="_3LWZlK").text.strip())
    except:
      phone_rating.append(np.nan)

    try:
      phone_num_ratings.append(phone.find_all('span',class_='_2_R_DZ')[0].find_all('span')[0].find_all('span')[0].text.strip())
    except:
      phone_num_ratings.append(np.nan)

    try:
      phone_price.append(phone.find("div",class_="_30jeq3 _1_WHN1").text.strip())
    except:
      phone_price.append(np.nan)


  df = pd.DataFrame({'phone_name':phone_name,'phone_price':phone_price,'phone_rating':phone_rating,'phone_num_rating':phone_num_ratings,'phone_img_url':phone_image_url})
  final_df = final_df.append(df)

  phone_image_url.clear()
  phone_name.clear()
  phone_rating.clear()
  phone_num_ratings.clear()
  phone_price.clear()

  time.sleep(2)


In [ ]:
final_df.shape

(120, 5)

In [ ]:
df.shape

(24, 5)

In [ ]:
final_df.isnull().sum()

phone_name          0
phone_price         0
phone_rating        2
phone_num_rating    2
phone_img_url       0
dtype: int64